# AgentWallet Protocol — Quickstart

**Autonomous financial infrastructure for AI agents on Solana.**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/YouthAIAgent/agentwallet/blob/master/examples/quickstart.ipynb)
[![PyPI](https://img.shields.io/pypi/v/aw-protocol-sdk)](https://pypi.org/project/aw-protocol-sdk/)
[![GitHub](https://img.shields.io/github/stars/YouthAIAgent/agentwallet)](https://github.com/YouthAIAgent/agentwallet)

In this notebook you'll:
1. Register & authenticate
2. Create an AI agent
3. Create a Solana wallet
4. Send a SOL transfer
5. Set up an escrow payment
6. Enforce spending policies
7. View analytics

**Time: ~5 minutes** | **Cost: Free (devnet)**

In [ ]:
!pip install -q aw-protocol-sdk httpx

## Setup

Connect to the AgentWallet API. Get your API key at [agentwallet.fun](https://agentwallet.fun).

In [ ]:
import asyncio
import httpx
from getpass import getpass

API_BASE = "https://api.agentwallet.fun/v1"

# Check API is alive
resp = httpx.get("https://api.agentwallet.fun/health")
print(f"API Status: {resp.json()}")

## 1. Register & Login

Create an account and get a JWT token.

In [ ]:
import secrets
import string

# Generate unique test credentials
suffix = secrets.token_hex(4)
EMAIL = f"demo-{suffix}@example.com"
PASSWORD = f"Demo{secrets.token_urlsafe(12)}!"
ORG_NAME = f"demo-org-{suffix}"

# Register
resp = httpx.post(f"{API_BASE}/auth/register", json={
    "email": EMAIL,
    "password": PASSWORD,
    "org_name": ORG_NAME,
})
print(f"Register: {resp.status_code}")

# Login
resp = httpx.post(f"{API_BASE}/auth/login", json={
    "email": EMAIL,
    "password": PASSWORD,
})
auth = resp.json()
TOKEN = auth["access_token"]
print(f"Logged in! Token: {TOKEN[:20]}...")

# Helper for authenticated requests
client = httpx.Client(
    base_url=API_BASE,
    headers={"Authorization": f"Bearer {TOKEN}"},
    timeout=30,
)

## 2. Create an AI Agent

Register your first autonomous agent with capabilities.

In [ ]:
agent = client.post("/agents", json={
    "name": f"quickstart-bot-{suffix}",
    "description": "Demo agent from quickstart notebook",
    "capabilities": ["payments", "escrow", "analytics"],
    "is_public": True,
}).json()

AGENT_ID = agent["id"]
print(f"Agent created!")
print(f"  ID:           {agent['id']}")
print(f"  Name:         {agent['name']}")
print(f"  Status:       {agent['status']}")
print(f"  Capabilities: {agent['capabilities']}")

## 3. Create a Solana Wallet

Every agent needs a wallet to hold and transfer SOL.

In [ ]:
wallet = client.post("/wallets", json={
    "agent_id": AGENT_ID,
    "wallet_type": "agent",
    "label": "primary-wallet",
}).json()

WALLET_ID = wallet["id"]
print(f"Wallet created!")
print(f"  ID:      {wallet['id']}")
print(f"  Address: {wallet['solana_address']}")
print(f"  Type:    {wallet['wallet_type']}")

# Check balance
balance = client.get(f"/wallets/{WALLET_ID}/balance").json()
print(f"  Balance: {balance.get('sol_balance', 0)} SOL")

## 4. Send a Transaction

Transfer SOL from the agent's wallet to any Solana address.

In [ ]:
# Note: This will work once the wallet is funded on devnet
# Use https://faucet.solana.com to airdrop devnet SOL

RECIPIENT = "CEQLGCWkpUjbsh5kZujTaCkFB59EKxmnhsqydDzpt6r6"  # Example address

try:
    tx = client.post("/transactions/transfer-sol", json={
        "from_wallet_id": WALLET_ID,
        "to_address": RECIPIENT,
        "amount_sol": 0.001,
        "memo": "quickstart-demo",
    }).json()
    print(f"Transaction created!")
    print(f"  TX ID:   {tx['id']}")
    print(f"  Status:  {tx['status']}")
    print(f"  Amount:  {tx['amount_sol']} SOL")
except Exception as e:
    print(f"Transaction created (wallet needs devnet funding to execute)")
    print(f"  Airdrop SOL: https://faucet.solana.com")

## 5. Escrow Payment

Create an escrow that holds SOL until conditions are met. Great for agent-to-agent payments.

In [ ]:
escrow = client.post("/escrow", json={
    "funder_wallet_id": WALLET_ID,
    "recipient_address": RECIPIENT,
    "amount_sol": 0.5,
    "conditions": {
        "task": "deliver sentiment analysis report",
        "deadline": "48 hours",
    },
    "expires_in_hours": 48,
}).json()

ESCROW_ID = escrow["id"]
print(f"Escrow created!")
print(f"  ID:         {escrow['id']}")
print(f"  Amount:     {escrow['amount_sol']} SOL")
print(f"  Status:     {escrow['status']}")
print(f"  Conditions: {escrow['conditions']}")

## 6. Spending Policy

Set guardrails so your agent can't overspend.

In [ ]:
policy = client.post("/policies", json={
    "agent_id": AGENT_ID,
    "policy_type": "spending_limit",
    "rules": {
        "max_per_transaction_sol": 1.0,
        "max_daily_sol": 10.0,
        "allowed_recipients": [RECIPIENT],
    },
}).json()

print(f"Policy created!")
print(f"  ID:         {policy['id']}")
print(f"  Type:       {policy['policy_type']}")
print(f"  Max/tx:     {policy['rules']['max_per_transaction_sol']} SOL")
print(f"  Max/day:    {policy['rules']['max_daily_sol']} SOL")

## 7. Analytics

Get real-time analytics for your agent's financial activity.

In [ ]:
analytics = client.get(f"/analytics/agents/{AGENT_ID}").json()

print(f"Agent Analytics")
print(f"  Total transactions: {analytics.get('total_transactions', 0)}")
print(f"  Total volume:       {analytics.get('total_volume_sol', 0)} SOL")
print(f"  Active escrows:     {analytics.get('active_escrows', 0)}")
print(f"  Active policies:    {analytics.get('active_policies', 0)}")

## Using the Python SDK

The same workflow using the official SDK (async/await):

In [ ]:
from agentwallet import AgentWallet

async def sdk_demo():
    # NOTE: SDK uses X-API-Key auth. For this demo we'll show the pattern.
    # In production, use your org's API key.
    async with AgentWallet(api_key="aw_live_YOUR_KEY", base_url=API_BASE) as aw:
        # List agents
        agents = await aw.agents.list(limit=5)
        print(f"Found {agents.total} agents")

        # List wallets
        wallets = await aw.wallets.list(limit=5)
        print(f"Found {wallets.total} wallets")

        # List escrows
        escrows = await aw.escrow.list(limit=5)
        print(f"Found {escrows.total} escrows")

print("SDK pattern (requires API key):")
print("  async with AgentWallet(api_key='aw_live_...') as aw:")
print("      agent = await aw.agents.create(name='my-bot')")
print("      wallet = await aw.wallets.create(agent_id=agent.id)")
print("      await aw.transactions.transfer_sol(...)")

---

## What's Next?

| Resource | Link |
|----------|------|
| API Docs | [api.agentwallet.fun/docs](https://api.agentwallet.fun/docs) |
| GitHub | [github.com/YouthAIAgent/agentwallet](https://github.com/YouthAIAgent/agentwallet) |
| Python SDK | `pip install aw-protocol-sdk` |
| Website | [agentwallet.fun](https://agentwallet.fun) |

### Advanced Features
- **ACP (Agent Commerce Protocol)**: Agent-to-agent commerce with negotiation + escrow
- **Swarms**: Orchestrate multi-agent task clusters
- **x402**: HTTP-native micropayments
- **PDA Wallets**: Program-derived addresses for on-chain control

### Framework Integrations
- **CrewAI**: `examples/crewai_example.py`
- **LangChain**: `examples/langchain_example.py`
- **AutoGen**: `examples/autogen_example.py`

---
*Built by [YouthAIAgent](https://github.com/YouthAIAgent) | Star us on GitHub!*